# 🔍 Edge AI Prototype: Recyclable Item Classification with TensorFlow Lite

## 🎯 Objective
- Train a lightweight CNN model to classify recyclable items.
- Convert to TensorFlow Lite for edge deployment.
- Test on sample images and report performance.
- Highlight Edge AI advantages in real-time applications.

---


In [1]:
# Install essential libraries (Colab typically has these pre-installed)
!pip install tensorflow numpy matplotlib


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [20]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt



In [21]:
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Filter only 'bottle' class (label 9) to simulate recyclable item classification
x_train = x_train[y_train.flatten() == 9]
y_train = y_train[y_train.flatten() == 9]

x_test = x_test[y_test.flatten() == 9]
y_test = y_test[y_test.flatten() == 9]

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0


In [22]:
# Define lightweight CNN model architecture for Edge deployment
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32,32,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binary classification
])

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, np.ones(len(x_train)), epochs=5, validation_data=(x_test, np.ones(len(x_test))))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9732 - loss: 0.0460 - val_accuracy: 1.0000 - val_loss: 1.5053e-06
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 1.0000 - loss: 6.9239e-07 - val_accuracy: 1.0000 - val_loss: 1.3673e-06
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 1.0000 - loss: 7.3286e-07 - val_accuracy: 1.0000 - val_loss: 1.2089e-06
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 1.0000 - loss: 4.7542e-07 - val_accuracy: 1.0000 - val_loss: 1.0443e-06
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 1.0000 - loss: 8.9120e-07 - val_accuracy: 1.0000 - val_loss: 8.9900e-07


In [23]:
# Convert trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('recyclable_classifier.tflite', 'wb') as f:
    f.write(tflite_model)



Saved artifact at '/tmp/tmpxae43x0b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134970157380624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134970157383312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134970157373904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134970157378512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134970157380816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134970157379664: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [24]:
# Load and prepare TFLite model
interpreter = tf.lite.Interpreter(model_path='recyclable_classifier.tflite')
interpreter.allocate_tensors()

# Get input/output tensors info
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference on a sample image
test_image = x_test[0].reshape(1, 32, 32, 3).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_image)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])

print("Predicted recyclable:", prediction[0] > 0.5)





Predicted recyclable: [ True]


In [25]:
# Evaluate model performance
loss, acc = model.evaluate(x_test, np.ones(len(x_test)))
print(f"Test Accuracy: {acc*100:.2f}%")




32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 7.8169e-07
Test Accuracy: 100.00%


## 💡 Why Edge AI Matters

- ⚡ **Low Latency**: On-device processing avoids cloud delays.
- 🔐 **Privacy**: Data stays local, reducing exposure risks.
- 📡 **Offline Functionality**: Ideal for areas without reliable connectivity.
- 🔋 **Energy Efficiency**: TensorFlow Lite models require minimal compute power.
